El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np 
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('car_data.csv')
data.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
238837,19/03/2016 03:49,8800,bus,2008,NaN,0,touran,100000,10,gasoline,volkswagen,NaN,18/03/2016 00:00,0,99086,02/04/2016 06:15
74137,06/03/2016 20:46,1490,convertible,2000,auto,163,other,150000,7,petrol,chrysler,no,06/03/2016 00:00,0,85521,07/03/2016 08:15
16827,08/03/2016 19:58,6500,wagon,2001,manual,131,a4,150000,10,gasoline,audi,no,08/03/2016 00:00,0,26441,05/04/2016 18:44
6701,19/03/2016 15:45,5399,sedan,2006,manual,80,golf,150000,9,petrol,volkswagen,no,19/03/2016 00:00,0,48429,07/04/2016 00:15
107317,03/04/2016 00:58,11500,coupe,2015,manual,100,corsa,30000,1,petrol,opel,no,03/04/2016 00:00,0,1156,07/04/2016 09:16
28266,10/03/2016 12:49,400,wagon,1997,manual,0,NaN,150000,0,petrol,opel,no,10/03/2016 00:00,0,26639,11/03/2016 12:47
166288,21/03/2016 20:43,2850,small,2004,manual,54,polo,80000,5,petrol,volkswagen,no,21/03/2016 00:00,0,86971,23/03/2016 01:46
100528,06/03/2016 15:44,9800,wagon,2009,manual,170,a4,150000,12,gasoline,audi,no,06/03/2016 00:00,0,53881,17/03/2016 14:46
259184,11/03/2016 16:50,6499,wagon,2006,NaN,163,3er,20000,6,gasoline,bmw,NaN,11/03/2016 00:00,0,13409,15/03/2016 08:45
308401,21/03/2016 12:55,1100,coupe,1999,manual,0,clk,150000,0,petrol,mercedes_benz,NaN,21/03/2016 00:00,0,52074,29/03/2016 21:17


In [3]:
data.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [3]:
data.info()
print()
display((data == 0).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

DateCrawled               0
Price                 10772
VehicleType               0
RegistrationYear          0
Gearbox                   0
Power                 40225
Model                     0
Mileage                   0
RegistrationMonth     37352
FuelType                  0
Brand                     0
NotRepaired               0
DateCreated               0
NumberOfPictures     354369
PostalCode                0
LastSeen                  0
dtype: int64

In [4]:
# Completar valores faltantes de las características categóricas con 'Unknown'
cat_features = ['VehicleType','Gearbox', 'FuelType', 'NotRepaired', 'Brand']
data[cat_features] = data[cat_features].fillna('Unknown')

# Remplazar potencia y precio cero con la media 
data['Power'] = data['Power'].replace(0, data['Power'].median())
data['Price'] = data['Price'].replace(0, data['Price'].median())

# Verificar valores
data.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
305865,08/03/2016 13:54,1800,Unknown,2017,manual,60,kadett,70000,10,Unknown,opel,no,08/03/2016 00:00,0,83109,12/03/2016 23:15
57672,16/03/2016 21:39,900,small,1998,manual,60,polo,150000,0,petrol,volkswagen,no,16/03/2016 00:00,0,68199,30/03/2016 18:15
157479,04/04/2016 17:44,320,small,1996,manual,105,NaN,150000,11,petrol,opel,Unknown,04/04/2016 00:00,0,51647,06/04/2016 19:17
139301,05/03/2016 21:37,300,small,1997,manual,75,almera,125000,0,petrol,nissan,Unknown,05/03/2016 00:00,0,47249,06/03/2016 04:17
143794,15/03/2016 22:32,13200,wagon,2005,auto,272,5er,150000,3,gasoline,bmw,no,14/03/2016 00:00,0,35390,29/03/2016 15:16
271742,04/04/2016 17:57,5999,bus,2007,manual,109,sprinter,150000,3,gasoline,mercedes_benz,no,04/04/2016 00:00,0,12587,06/04/2016 19:48
292523,09/03/2016 19:39,5100,sedan,2008,manual,109,i_reihe,100000,12,petrol,hyundai,no,09/03/2016 00:00,0,26871,03/04/2016 13:47
51452,04/04/2016 18:48,1500,small,2005,manual,111,rio,150000,11,petrol,kia,yes,04/04/2016 00:00,0,67063,06/04/2016 21:15
29926,29/03/2016 18:47,3000,bus,2007,manual,90,transit,125000,3,gasoline,ford,no,29/03/2016 00:00,0,25821,29/03/2016 18:47
50674,25/03/2016 18:54,8250,sedan,2007,manual,105,golf,150000,11,gasoline,volkswagen,no,25/03/2016 00:00,0,94209,07/04/2016 01:18


In [5]:
#Eliminando columnas innecesarias
columns_to_drop = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode', 'RegistrationMonth', 'Model']
data = data.drop(columns=columns_to_drop)

# Verificar valores
data.sample(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Mileage,FuelType,Brand,NotRepaired
46515,2200,convertible,2003,manual,136,150000,petrol,peugeot,no
295926,4300,convertible,2002,manual,109,70000,petrol,peugeot,no
205469,2750,wagon,2001,manual,105,150000,petrol,volkswagen,no
53203,5800,coupe,2001,auto,218,150000,petrol,mercedes_benz,no
163376,2795,wagon,1998,manual,101,150000,petrol,volkswagen,no
88154,2700,sedan,1996,manual,125,150000,petrol,audi,no
241790,6300,sedan,2001,manual,271,150000,petrol,subaru,no
318414,3399,wagon,2001,manual,170,150000,petrol,bmw,no
290075,900,bus,2001,manual,105,150000,petrol,renault,yes
277567,9999,small,2011,auto,179,100000,petrol,volkswagen,no


In [6]:
# Convertir características categóricas a variables dummy
data = pd.get_dummies(data, columns=cat_features, drop_first=True)


## Entrenamiento del modelo 

In [7]:
# Definir característica y objetivo 
X = data.drop(['Price'],axis=1)
y = data['Price']

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
# Medir el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo de Regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predicción
y_pred = linear_model.predict(X_valid)

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f'Regresión Lineal RMSE: {rmse}')
print(f'Tiempo de entrenamiento: {time.time() - start_time} segundos')

Regresión Lineal RMSE: 3337.243399599282
Tiempo de entrenamiento: 0.9203827381134033 segundos


In [9]:
# Definir el modelo Random Forest con menos estimadores y menos profundidad
rf_model_fast = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
start_time = time.time()


# Entrenar el modelo rápido
rf_model_fast.fit(X_train, y_train)
train_time = time.time() - start_time

# Predicción en conjunto de prueba
y_pred_rf_fast = rf_model_fast.predict(X_valid)

# Calcular el RMSE
rmse_rf_fast = np.sqrt(mean_squared_error(y_valid, y_pred_rf_fast))
print(f'Bosque Aleatorio (rápido) RMSE: {rmse_rf_fast}')
print(f'Tiempo de entrenamiento: {time.time() - start_time} segundos')

Bosque Aleatorio (rápido) RMSE: 2015.0908268480136
Tiempo de entrenamiento: 32.78172779083252 segundos


In [10]:
# Definir el modelo CatBoot
cat_model = CatBoostRegressor(loss_function='RMSE', iterations=100, depth=10, learning_rate=0.1, verbose=10)

# Medir el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
cat_model.fit(X_train, y_train, cat_features=[])

# Predicción
y_pred = cat_model.predict(X_valid)

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f'CatBoost RMSE: {rmse}')
print(f'Tiempo de entrenamiento: {time.time() - start_time} segundos')

0:	learn: 4146.4313741	total: 107ms	remaining: 10.6s
10:	learn: 2578.0178710	total: 405ms	remaining: 3.27s
20:	learn: 2153.3010627	total: 719ms	remaining: 2.71s
30:	learn: 2003.9091615	total: 1.04s	remaining: 2.32s
40:	learn: 1940.6965711	total: 1.36s	remaining: 1.96s
50:	learn: 1901.2951811	total: 1.69s	remaining: 1.62s
60:	learn: 1875.4726862	total: 2.02s	remaining: 1.29s
70:	learn: 1852.9761940	total: 2.36s	remaining: 964ms
80:	learn: 1834.5280150	total: 2.69s	remaining: 632ms
90:	learn: 1818.9198013	total: 3.02s	remaining: 298ms
99:	learn: 1806.5226667	total: 3.33s	remaining: 0us
CatBoost RMSE: 1847.7315314512884
Tiempo de entrenamiento: 3.520310401916504 segundos


In [11]:
# XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=10, learning_rate=0.1)

# Medir el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
xgb_model.fit(X_train, y_train)

# Predicción
y_pred = xgb_model.predict(X_valid)

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f'XGBoost RMSE: {rmse}')
print(f'Tiempo de entrenamiento: {time.time() - start_time} segundos')

XGBoost RMSE: 1737.4264993527702
Tiempo de entrenamiento: 4.138355016708374 segundos


## Conclusiones

### Presición del modelo 

- Mejor modelo : XGBoost con un RSME de 1700.72
- Peor modelo : Regresión lineal conun RSME de 3155.28
- Los modelos Random Forest, CatBoost, y XGBoost superaron signficativamente a la regresión lineal en terminos de precisión 

### Tiempo de Entrenamiento

- Más rápido: CatBoost con un tiempo de entrenamiento de 4.12 segundos
- Más lento: Bosque aleatorio con un tiempo de 107.00 segundos 
- El bosque aleatorio tomó mucho más tiempo para entrenar comparado con otros modelos. 

En resumen el mejor modelo teniendo un balance entre precisión y velocidad es CatBoost que ofrece una buena precisión (RSME: 1832.09) y un tiempo de entrenamiento muy rápido (4.12 segundos). Esto lo convierte en una opción atractiva cuando se considera tanto calidad de la predicción como la velocidad de entrenamiento. 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos